## Paso 1 : Preprocesamiento de los datos
1. Cree la columna 'Age' a partir de la columna 'Year'.
Asuma que el año actual es 2021.
2. Elimine las columnas 'Year' y 'Car_Name'.

In [267]:
# Leer Dataset

import pandas as pd

train = pd.read_csv("../files/input/train_data.csv.zip", index_col = False, compression = "zip")
test = pd.read_csv("../files/input/test_data.csv.zip", index_col = False, compression = "zip")

In [268]:
train.head()

,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner
0,jazz,2016,7.40,8.500,15059,Petrol,Dealer,Automatic,0
1,i10,2013,4.00,4.600,30000,Petrol,Dealer,Manual,0
2,TVS Apache RTR 180,2011,0.50,0.826,6000,Petrol,Individual,Manual,0
3,eon,2016,3.15,4.430,15000,Petrol,Dealer,Manual,0
4,Royal Enfield Thunder 350,2013,1.25,1.500,15000,Petrol,Individual,Manual,0


In [269]:
# Año de referencia
reference_year = 2021

# Crear columna Age en ambos sets
train['Age'] = reference_year - train['Year']
test['Age'] = reference_year - test['Year']

In [270]:
test.head()

,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Age
0,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0,8
1,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0,4
2,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0,10
3,ciaz,2015,6.75,8.12,18796,Petrol,Dealer,Manual,0,6
4,s cross,2015,6.50,8.61,33429,Diesel,Dealer,Manual,0,6


In [271]:
a= test['Owner'].unique()
print(a)

[0 1]


In [272]:
# Remueva la columna "Year" y "Car_Name"
train.drop(columns=["Year","Car_Name"], inplace=True)
test.drop(columns=["Year","Car_Name"], inplace=True)

In [273]:

test.head()

,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Age
0,4.75,9.54,43000,Diesel,Dealer,Manual,0,8
1,7.25,9.85,6900,Petrol,Dealer,Manual,0,4
2,2.85,4.15,5200,Petrol,Dealer,Manual,0,10
3,6.75,8.12,18796,Petrol,Dealer,Manual,0,6
4,6.50,8.61,33429,Diesel,Dealer,Manual,0,6


## Paso 2 
1. Divida los datasets en x_train, y_train, x_test, y_test.

In [274]:
x_train = train.drop(columns=['Selling_Price'])
y_train = train['Selling_Price']

x_test = test.drop(columns=['Selling_Price'])
y_test = test['Selling_Price']

In [275]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Present_Price  211 non-null    float64
 1   Driven_kms     211 non-null    int64  
 2   Fuel_Type      211 non-null    object 
 3   Selling_type   211 non-null    object 
 4   Transmission   211 non-null    object 
 5   Owner          211 non-null    int64  
 6   Age            211 non-null    int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 11.7+ KB


In [266]:
y_test.head()

0    4.75
1    7.25
2    2.85
3    6.75
4    6.50
Name: Selling_Price, dtype: float64

## Paso 3 : Cree un pipeline para el modelo de clasificación
Este pipeline debe contener las siguientes capas:
1. Transforma las variables categoricas usando el método one-hot-encoding.
2. Escala las variables numéricas al intervalo [0, 1].
3. Selecciona las K mejores entradas.
4. Ajusta un modelo de regresion lineal.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

# Identificar columnas categóricas y numéricas
categoricas = ['Fuel_Type', 'Selling_type', 'Transmission']

# Preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categoricas),
        ('num', 'passthrough', ['Present_Price'])
    ],
    remainder='passthrough'
)

#Selección de las k mejores características
selectkbest = SelectKBest(score_func=f_regression, k=10)

LinR= LinearRegression()

#Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', MinMaxScaler()),
    ('feature_selection', selectkbest), 
    ('classifier', LinR)  
])

# Paso 4: Optimización
1. Optimice los hiperparametros del pipeline usando validación cruzada.
2. Use 10 splits para la validación cruzada. 
3. Use el error medio absoluto para medir el desempeño modelo.

In [241]:

from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_absolute_error
import numpy as np

CVkf = KFold(n_splits=10, shuffle=True, random_state=42)

param_grid = {
    'classifier__fit_intercept': [True, False]
}

model=GridSearchCV(
    pipeline,
    param_grid,
    cv=CVkf,
    scoring= "neg_mean_absolute_error",
    n_jobs=-1,
    verbose=3,
    refit=True
    )

# Entrenar el modelo
model.fit(x_train, y_train)


# Predicciones en el conjunto de prueba
y_pred = model.predict(x_test)


# Calcular el Error Medio Absoluto (MAE) en el conjunto de prueba
mae_test = mean_absolute_error(y_test, y_pred)

# Resultados
print(f"Mejores parámetros: {model.best_params_}")
print(f"Mejor MAE (validación cruzada): {-model.best_score_:.4f}")
print(f"MAE en el conjunto de prueba: {mae_test:.4f}")


Fitting 10 folds for each of 2 candidates, totalling 20 fits
Mejores parámetros: {'classifier__fit_intercept': True}
Mejor MAE (validación cruzada): 1.1350
MAE en el conjunto de prueba: 1.4542


In [242]:

print(model.score(x_train, y_train), model.score(x_test, y_test))

-1.0504276890529465 -1.454180285946514


## Paso 5 : Guardar el modelo
Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

In [243]:
import gzip
import pickle
import os

models_dir = '../files/models'
os.makedirs(models_dir, exist_ok=True)
output_path = "../files/models/model.pkl.gz"
with gzip.open(output_path, 'wb') as file:
    pickle.dump(model, file)

## Paso 6.
1. Calcule las metricas r2, error cuadratico medio, y error absoluto medio para los conjuntos de entrenamiento y prueba. Guardelas en el archivo
files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo. Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba. Por ejemplo:

{'type': 'metrics', 'dataset': 'train', 'r2': 0.8, 'mse': 0.7, 'mad': 0.9}
{'type': 'metrics', 'dataset': 'test', 'r2': 0.7, 'mse': 0.6, 'mad': 0.8}

In [244]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Realizar las predicciones sobre los conjuntos de entrenamiento y prueba
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

# Calcular métricas para el conjunto de entrenamiento
r2_train = r2_score(y_train, y_train_pred)
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)


# Calcular métricas para el conjunto de prueba
r2_test = r2_score(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Resultados
print("Métricas para el conjunto de entrenamiento:")
print(f"R^2: {r2_train:.4f}")
print(f"Error Cuadrático Medio (MSE): {mse_train:.4f}")
print(f"Error Absoluto Medio (MAE): {mae_train:.4f}")

print("\nMétricas para el conjunto de prueba:")
print(f"R^2: {r2_test:.4f}")
print(f"Error Cuadrático Medio (MSE): {mse_test:.4f}")
print(f"Error Absoluto Medio (MAE): {mae_test:.4f}")


Métricas para el conjunto de entrenamiento:
R^2: 0.8977
Error Cuadrático Medio (MSE): 2.3640
Error Absoluto Medio (MAE): 1.0504

Métricas para el conjunto de prueba:
R^2: 0.7898
Error Cuadrático Medio (MSE): 6.7090
Error Absoluto Medio (MAE): 1.4542


#### Guardar las métricas en un archivo json

In [245]:
import json

metrics = [
    {
        "type": "metrics",
        'dataset': 'train',
        'r2': r2_train,
        'mse': mse_train,
        'mad': mae_train,
    },
    {
        "type": "metrics",
        'dataset': 'test',
        'r2': r2_test,
        'mse': mse_test,
        'mad': mae_test,
    }
]

output_dir = "../files/output"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir,"metrics.json")

with open(output_path,"w") as file:
        file.write(json.dumps(metrics) + '\n')